In [1]:

from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import numpy as np
import zipfile

import csv
import urllib3
import os
import shutil

In [2]:
# Open url for scraping. All data sources are stored within a table. All target data files have the
# same naming scheme, we will take advantage of this

In [3]:

url = 'http://www.fueleconomy.gov/feg/download.shtml'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')


reg = 'epadata/\d{2}data.zip' ##luckily our target has a unique identifier for href
suf = 'http://www.fueleconomy.gov/feg/'

In [4]:
# Find target data sources within html page

In [5]:
links = soup.find_all('a')
zlist = []
for tag in links:
    link = tag.get('href',None)
    if link is not None:
        if re.match(reg, link):
            zlist.append(link)

In [6]:
# Download data to current working directory. These files are zipped. Unzip files.

In [7]:
filelst = []
for x in zlist:
    url = suf+x
    localfn = url.split('/')[-1]
    yearno = int(localfn[:2])
    if yearno < 50 and yearno > 00:
        r = requests.get(url, stream=True)
        with open(localfn, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
        zip_ref = zipfile.ZipFile(localfn, 'r')
        lcl = "dldir"+localfn
        filelst.append(lcl)
        zip_ref.extractall(lcl)
        zip_ref.close()
       
        


In [8]:
# create directory to store data

In [9]:
datdir = "shibe_data"
if not os.path.exists(datdir):
    os.makedirs(datdir)

In [10]:
# extract downloaded data to single directory. Rename files so that they have uniform names

In [11]:
reg = "(.xlsx|.xls|.csv)$"
for item in filelst:
    for filename in os.listdir(item):
        curname =  item + "\\" + filename 
        
        
        m = re.search(reg, filename)
        if m:
            newname = datdir + "\\" + item[:-4] + m.group(0)
            os.rename(curname, newname)


In [12]:
# clean up files and directories created while scraping data

In [13]:

lst = os.listdir(os.getcwd())
for x in lst:
    m = re.search(".zip$", x)
    if m:
        n = re.search("^dldir",x)
        if n:
            shutil.rmtree(x)
        else:
            os.remove(x)
